In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import numpy as np
import pandas as pd
#np.random.seed(0) # keep this the same
#SPECIES = ['yb', 'er', 'gd']


def generate_recipe(num_trials, species, num_layer = 4, Rmin=5, Rmax=34):
    recipes = []
    header = []
    num_elem = len(species)
    for i in range(num_trials):
        recipe = np.array([])
        for j in range(num_layer):
            total_c = np.random.uniform() # 0 < total concentration < 1
            proportion = np.random.dirichlet(np.ones(num_elem)) * total_c # proportion of elements
            recipe = np.append(recipe, proportion)
            if len(header) < num_elem * num_layer:
                for specie in species:
                    header.append(f"{specie}_{j+1}")
        radius = np.random.uniform(Rmin,Rmax)
        recipe = np.append(recipe, radius)
        if len(header) < num_elem * num_layer+1:
            header.append("radius")
        recipes.append(recipe)

    print("features:", header)
    df = pd.DataFrame(recipes, columns = header)
    
    return df

In [7]:
np.random.seed(0)
SPECIES = ['yb', 'er', 'tm']
df_test = generate_recipe(20, SPECIES, num_layer = 2)
df_test

features: ['yb_1', 'er_1', 'tm_1', 'yb_2', 'er_2', 'tm_2', 'radius']


,yb_1,er_1,tm_1,yb_2,er_2,tm_2,radius
0,0.232363,0.170808,0.145642,0.114620,0.063541,0.245493,32.946220
1,0.190313,0.091302,0.101826,0.368123,0.455430,0.102044,29.145976
2,0.211952,0.399449,0.166755,0.262722,0.021806,0.176951,9.157245
3,0.440953,0.320074,0.183642,0.321150,0.443082,0.010002,22.911429
4,0.117877,0.353542,0.140676,0.112699,0.234625,0.012184,24.336235
5,0.210316,0.122899,0.337423,0.052266,0.035734,0.275711,7.959300
6,0.024067,0.144849,0.039960,0.229147,0.141544,0.095620,24.033558
7,0.012606,0.026494,0.099082,0.031166,0.001730,0.064205,18.590885
8,0.392140,0.567737,0.016884,0.059799,0.163989,0.059020,14.221512
9,0.013194,0.234674,0.166395,0.115825,0.015425,0.134140,31.949590


In [8]:
df_test.to_csv('YbErTm_initial_features_20NPs.csv', sep=',')

In [9]:
from NanoParticleTools.flows.flows import get_npmc_flow
from NanoParticleTools.inputs.nanoparticle import SphericalConstraint
from fireworks import LaunchPad
from maggma.stores.mongolike import MongoStore
from jobflow import JobStore
from jobflow.managers.fireworks import flow_to_workflow
import uuid

lp_file = '/Users/xiaojing/my_launchpad.yaml'
lp = LaunchPad.from_file(lp_file)

docs_store = MongoStore.from_launchpad_file(lp_file, 'test_fws_npmc')
data_store = MongoStore.from_launchpad_file(lp_file, 'test_docs_npmc')
store = JobStore(docs_store, additional_stores={'trajectories': data_store})



excitation_wavelength = 980
excitation_power = 1e5

npmc_args = {'npmc_command': 'NPMC',
             'num_sims': 4,
             'base_seed': 1000,
             'thread_count': 4,
             #'simulation_length': 100000 #
             'simulation_time': 0.01 #10ms
             }
spectral_kinetics_args = {'excitation_wavelength': excitation_wavelength,
                          'excitation_power': excitation_power}

initial_state_db_args = {'interaction_radius_bound': 3}

np_uuid = str(uuid.uuid4())

for i in np.arange(0,20):
    # constraints = [SphericalConstraint(50)]
    # dopant_specifications = [(0, 0.2, 'Yb', 'Y'),
    #                          (0, 0.02, 'Er', 'Y')]
    constraints = [SphericalConstraint(df_test['radius'][i]), SphericalConstraint(34),SphericalConstraint(40)]
    dopant_specifications = [(0, df_test['yb_1'][i], 'Yb', 'Y'),
                             (0, df_test['er_1'][i], 'Er', 'Y'),
                             (0, df_test['tm_1'][i], 'Tm', 'Y'),
                             (1, df_test['yb_2'][i], 'Yb', 'Y'),
                             (1, df_test['er_2'][i], 'Er', 'Y'), 
                             (1, df_test['tm_2'][i], 'Tm', 'Y'),
                             (2, 0.3, 'Surface6', 'Y')]
    for doping_seed in range(4):
        flow = get_npmc_flow(constraints=constraints,
                             dopant_specifications=dopant_specifications,
                             doping_seed=doping_seed,
                             spectral_kinetics_args=spectral_kinetics_args,
                             initial_state_db_args=initial_state_db_args,
                             npmc_args=npmc_args,
                             output_dir='./scratch'
                             )

        wf = flow_to_workflow(flow, store=store)
        lp.add_wf(wf)

/opt/anaconda3/lib/python3.9/site-packages/maggma-0.44.1-py3.9.egg/maggma/utils.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2022-08-05 20:00:27,068 INFO Added a workflow. id_map: {-1: 7463}
2022-08-05 20:00:27,268 INFO Added a workflow. id_map: {-2: 7464}
2022-08-05 20:00:27,438 INFO Added a workflow. id_map: {-3: 7465}
2022-08-05 20:00:27,560 INFO Added a workflow. id_map: {-4: 7466}
2022-08-05 20:00:27,889 INFO Added a workflow. id_map: {-5: 7467}
2022-08-05 20:00:28,052 INFO Added a workflow. id_map: {-6: 7468}
2022-08-05 20:00:28,293 INFO Added a workflow. id_map: {-7: 7469}
2022-08-05 20:00:28,537 INFO Added a workflow. id_map: {-8: 7470}
2022-08-05 20:00:28,708 INFO Added a workflow. id_map: {-9: 7471}
2022-08-05 20:00:29,020 INFO Added a workflow. id_map: {-10: 7472}
2022-08-05 20:00:29,164 INFO Added a workflow. id_map: {-11: 7473}
2022-08-05 20:00:29,403 INFO Added a workflow. id_map: {-12: 7474}
2022-08-05 20:00:29,603 INFO Added a workflow. id_map: {-13: 7475}
2022-08-05 20:00:29,922 INFO Added a workflow. id_map: {-14: 7476}
2022-08-05 20:00:30,107 INFO Added a workflow. id_map: {-15: 7477}
2022

In [81]:
df_test.to_csv('YbEr3shell1ligandFeatures_seed=1_100trials_220222.csv', sep=',')

##### import pandas as pd
import numpy as np
import math
from NanoParticleTools.species_data.species import Dopant
from maggma.stores.mongolike import MongoStore
from matplotlib import pyplot as plt
from bson import ObjectId
import time

from utils import get_int, get_qe

In [ ]:
df_test.to_csv('YbEr_0111_seed=2_100trials_220225.csv', sep=',')

In [4]:
np.random.seed(3)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test['yb_1'] = 0
df_test['er_1'] = 0
df_test['yb_4'] = 0
df_test['er_4'] = 0
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0,0,0.253612,0.257216,0.102308,0.023277,0,0
1,0,0,0.017021,0.573842,0.159868,0.099385,0,0
2,0,0,0.033574,0.188384,0.315940,0.660055,0,0
3,0,0,0.101697,0.123358,0.097072,0.172163,0,0
4,0,0,0.178768,0.022611,0.129473,0.354113,0,0
...,...,...,...,...,...,...,...,...
95,0,0,0.055713,0.503747,0.091283,0.488495,0,0
96,0,0,0.099821,0.791426,0.333766,0.401012,0,0
97,0,0,0.399950,0.558708,0.093394,0.462172,0,0
98,0,0,0.632850,0.008745,0.432885,0.438277,0,0


In [5]:
df_test.to_csv('YbEr_0110_seed=3_100trials_220226.csv', sep=',')

In [6]:
np.random.seed(4)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test['yb_2'] = 0
df_test['er_2'] = 0
df_test['yb_3'] = 0
df_test['er_3'] = 0
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.174439,0.792591,0,0,0,0,0.379487,0.055305
1,0.826891,0.036102,0,0,0,0,0.411914,0.537064
2,0.013544,0.159621,0,0,0,0,0.019409,0.088785
3,0.348275,0.176129,0,0,0,0,0.401341,0.101192
4,0.167943,0.386798,0,0,0,0,0.331127,0.100695
...,...,...,...,...,...,...,...,...
95,0.087373,0.623423,0,0,0,0,0.156276,0.665805
96,0.515655,0.088364,0,0,0,0,0.793022,0.006704
97,0.058850,0.667478,0,0,0,0,0.223525,0.292254
98,0.662593,0.055272,0,0,0,0,0.136721,0.020390


In [7]:
df_test.to_csv('YbEr_1001_seed=4_100trials_220226.csv', sep=',')

In [8]:
np.random.seed(5)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test['yb_2'] = 0
df_test['er_2'] = 0
df_test['yb_4'] = 0
df_test['er_4'] = 0
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.199420,0.022573,0,0,0.516842,0.249066,0,0
1,0.033182,0.408127,0,0,0.305755,0.323033,0,0
2,0.151437,0.176127,0,0,0.003884,0.020423,0,0
3,0.000682,0.000961,0,0,0.869730,0.000753,0,0
4,0.284175,0.529558,0,0,0.008861,0.102593,0,0
...,...,...,...,...,...,...,...,...
95,0.762830,0.163430,0,0,0.066405,0.879484,0,0
96,0.058558,0.265485,0,0,0.762293,0.122715,0,0
97,0.519173,0.122529,0,0,0.498914,0.079554,0,0
98,0.347064,0.530796,0,0,0.143052,0.078969,0,0


In [9]:
df_test.to_csv('YbEr_1010_seed=5_100trials_220227.csv', sep=',')

In [10]:
np.random.seed(6)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.169505,0.723355,0.004666,0.037031,0.302251,0.227566,0.188109,0.434411
1,0.235608,0.282428,0.755315,0.234910,0.848876,0.027392,0.394233,0.324405
2,0.605448,0.103684,0.649821,0.307826,0.678399,0.038877,0.281748,0.389561
3,0.559705,0.377645,0.207653,0.194819,0.358836,0.047272,0.349487,0.024322
4,0.200987,0.197386,0.070398,0.213462,0.289134,0.045672,0.218783,0.112321
...,...,...,...,...,...,...,...,...
95,0.491459,0.304527,0.327239,0.030525,0.081630,0.024743,0.023739,0.253247
96,0.436692,0.087810,0.400239,0.225345,0.636502,0.304421,0.247044,0.579800
97,0.249974,0.666919,0.702662,0.160439,0.402745,0.175821,0.501424,0.008427
98,0.026757,0.029923,0.187657,0.415539,0.049884,0.453898,0.093214,0.000577


In [11]:
df_test.to_csv('YbEr_1111_seed=6_100trials_220302.csv', sep=',')

In [3]:
np.random.seed(7)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.055249,0.021059,0.601572,0.121893,0.096741,0.404379,0.205538,0.294345
1,0.063672,0.317269,0.259370,0.650223,0.024902,0.906304,0.235311,0.714818
2,0.146971,0.762157,0.477855,0.272555,0.121193,0.083656,0.095584,0.381817
3,0.236095,0.532552,0.160308,0.115741,0.282340,0.375060,0.132060,0.587265
4,0.128333,0.052119,0.245722,0.180731,0.006031,0.408855,0.272512,0.436882
...,...,...,...,...,...,...,...,...
95,0.081376,0.474624,0.273876,0.570246,0.020836,0.398528,0.098286,0.669888
96,0.114187,0.438297,0.176533,0.496036,0.443462,0.470181,0.017235,0.006218
97,0.118622,0.001546,0.073034,0.056023,0.368257,0.060772,0.178668,0.027952
98,0.427922,0.261156,0.106383,0.774826,0.606281,0.296660,0.327771,0.326647


In [4]:
df_test.to_csv('YbEr_1111_seed=7_100trials_220304_testDataset.csv', sep=',')

In [5]:
np.random.seed(8)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.550007,0.323423,0.508836,0.022020,0.176659,0.253810,0.243181,0.235210
1,0.428475,0.332420,0.036478,0.389613,0.261643,0.072131,0.255816,0.727055
2,0.023927,0.047016,0.082442,0.813703,0.013639,0.015093,0.671562,0.092599
3,0.070578,0.361897,0.045574,0.592665,0.040419,0.755155,0.660665,0.129507
4,0.028770,0.010142,0.374371,0.252684,0.039959,0.496185,0.807537,0.132114
...,...,...,...,...,...,...,...,...
95,0.395216,0.078388,0.086228,0.030623,0.560711,0.216733,0.132990,0.830126
96,0.106813,0.272342,0.234682,0.166854,0.013922,0.081974,0.026464,0.686270
97,0.101942,0.681991,0.311623,0.533322,0.026049,0.937616,0.131727,0.181196
98,0.072635,0.261064,0.082408,0.214002,0.031673,0.235641,0.143377,0.082162


In [6]:
df_test.to_csv('YbEr_1111_seed=8_100trials_220304.csv', sep=',')

In [7]:
np.random.seed(9)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.005233,0.005141,0.051297,0.082532,0.319980,0.098528,0.027518,0.317981
1,0.030295,0.920669,0.386595,0.186165,0.390462,0.157376,0.510717,0.183722
2,0.267542,0.672982,0.034253,0.135383,0.232592,0.416948,0.022037,0.163505
3,0.697122,0.209155,0.268374,0.195161,0.104524,0.373450,0.091628,0.095730
4,0.034425,0.890775,0.586431,0.051623,0.004052,0.034519,0.041541,0.121065
...,...,...,...,...,...,...,...,...
95,0.853080,0.046719,0.356975,0.050725,0.403655,0.115202,0.151641,0.279951
96,0.058273,0.198301,0.063413,0.873498,0.057559,0.484992,0.288892,0.357213
97,0.078057,0.051965,0.303089,0.615523,0.036483,0.052314,0.141581,0.142707
98,0.024768,0.158220,0.370293,0.266757,0.120963,0.097490,0.556184,0.094455


In [ ]:
df_test.to_csv('YbEr_1111_seed=9_100trials_220307.csv', sep=',')

In [9]:
np.random.seed(10)
SPECIES = ['yb', 'er']
df_test = generate_recipe(100, SPECIES, num_layer = 4)
df_test

features: ['yb_1', 'er_1', 'yb_2', 'er_2', 'yb_3', 'er_3', 'yb_4', 'er_4']


,yb_1,er_1,yb_2,er_2,yb_3,er_3,yb_4,er_4
0,0.015778,0.755542,0.546995,0.201809,0.175337,0.022726,0.024193,0.064147
1,0.001185,0.002764,0.482382,0.130144,0.565210,0.352564,0.041825,0.100345
2,0.164272,0.277562,0.273770,0.239368,0.189678,0.615545,0.066893,0.252343
3,0.110963,0.003021,0.198380,0.427907,0.162682,0.036265,0.105737,0.648911
4,0.086482,0.239029,0.085335,0.008126,0.152347,0.231767,0.391512,0.064793
...,...,...,...,...,...,...,...,...
95,0.140108,0.052357,0.030114,0.000184,0.001065,0.011146,0.186021,0.200258
96,0.310118,0.137618,0.521743,0.290158,0.065750,0.496879,0.107098,0.022311
97,0.080335,0.745199,0.240799,0.742904,0.012952,0.186787,0.056836,0.050179
98,0.007555,0.050511,0.161524,0.217632,0.123929,0.160346,0.257067,0.466227


In [10]:
df_test.to_csv('YbEr_1111_seed=10_100trials_220307.csv', sep=',')